![alt text](Imagens/7Labs.png "Title")

Esse código foi criado para realizar a coletar de dados de energia e água presentes nas planilhas de controle de dados da Utilidades e Meio Ambiente da Cervejaria Sete Lagoas no período de 2017 à 2020.

Primeiramente, realizamos a instalação e o import das bibliotecas necessárias

In [1]:
!pip install pandas
!pip install matplotlib
!pip install xlrd
!pip install openpyxl

In [2]:
import pandas as pd
import numpy as np
import h5py
import matplotlib.pyplot as plt
import datetime

Configuramos alguns parâmetros de exibição de imagens e tabelas.
Além disso, configuramos algumas variáveis necessárias para automatização do fluxo de coleta

In [3]:
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

pd.set_option('max_rows', None)
pd.set_option('max_columns', None)

Files_Folder = 'Dados/'
Ano = ['2017/','2018/','2019/','2020/','2021/']

Realiza a leitura de todos os dados de um determinado ano.

Compara os nomes das colunas para entender a formação das estruturas de dados

*Interessante buscar uma forma de carregar apenas as linhas iniciais do DataFrame para não demorar demais no processamento*

In [4]:
Files_Folder = 'Dados MA/'
Ano = ['2017','2018','2019','2020','2021']
Mes = ['Jan','Fev','Mar','Abr','Mai','Jun','Jul','Ago','Set','Out','Nov','Dez']

my_data_y_MA = pd.DataFrame()

for j in range(0,4):
    for i in range(0,12):
        print(i)
        Files_path = Files_Folder +  'Água indice ' + Ano[j] +'.xlsx'
        try:
            df_y_MA = pd.read_excel(Files_path, sheet_name=Mes[i], skiprows=[0,1,2,4])
            df_y_MA_filter = df_y_MA[['Dia','Consumo de Água (m³).1']].copy()

            my_data_y_MA = my_data_y_MA.append(df_y_MA_filter, ignore_index = True)
        except Exception as e:
            print(e)
            #            pass

0
1
2
3
4
5
6
7
8
9
10
11
0
1
2
3
4
5
6
7
8
9
10
11
0
1
2
3
4
5
6
7
8
9
10
11
0
1
2
3
4
5
6
7
8
9
10
11


Salve a tabela em um arquivo chamado DataBase

In [5]:
my_data_y_MA
my_data_y_MA = my_data_y_MA.dropna()

index = my_data_y_MA[(my_data_y_MA['Dia']=='ACUM') | (my_data_y_MA['Dia']=='Meta acum') | (my_data_y_MA['Dia']=='VALOR REAL') | (my_data_y_MA['Dia']=='BASE HANNA')].index.values
my_data_y_MA.drop(index, inplace = True)

my_data_y_MA.to_excel(Files_Folder + 'DataBase.xlsx', index = False)

In [ ]:
my_data_y_all = my_data_y.merge(my_data_y_MA, on='Dia', how='outer')
my_data = my_data_y_all.merge(my_data_x, on='Dia', how='outer')
sorted_df = my_data.sort_values(by='Dia')
sorted_df = sorted_df.dropna()
sorted_df

In [ ]:
Files_Folder = 'Dados/'
path = Files_Folder + 'DataBase.xlsx'
dados = pd.read_excel(path)
dados = dados.sort_values("Dia", ascending=True)

In [ ]:
my_list_y = pd.DataFrame()
my_list_x = pd.DataFrame()
my_data_y = pd.DataFrame()
my_data_x = pd.DataFrame()

for j in range(0,5):
    for i in range(1,13):
        Files_path = Files_Folder + Ano[j] + '/Planilha de Dados ' + str(i) +'.xlsx'
    
        try:
            # Coleta informações das variáveis dependentes
            df_y = pd.read_excel(Files_path, sheet_name='EE_Dados', skiprows=[0,1,3])
            df_y_filter = df_y[['Dia','PRINCIPAL (kW)']].copy()
        
            # Coleta informações das variáveis independentes
            df_x = pd.read_excel(Files_path, sheet_name='Prod. Líquida', skiprows=[0,1])
            df_x_filter = df_x[['Dia','L501 (696)','L502 (697)','L503 (698)','L511 (699)','L512 (700)','L561 (927)',
                                'L562 (928)','563 (964)']].copy().fillna(0)

            # Coleta informações das variáveis produção Mosto e CO2
    #        df_CO2 = pd.read_excel(Files_path, sheet_name='CO2_Dados', skiprows=[0,1,3])
    #        df_CO2_filter = df_CO2[['Consumo - Balanceamento de massa         (Kg)','Volume de Mosto Corrigido para 12 P diário SALA 1Cod,MÊS 72146', Volume de Mosto Corrigido para 12 P diário  SALA 2 Cod,MÊS 7214']].copy()

            my_data_y = my_data_y.append(df_y_filter, ignore_index = True)
            my_data_x = my_data_x.append(df_x_filter, ignore_index = True)
        except Exception as e:
            pass

Código para filtrar DataFrame baseado em localização de um valor específico

In [ ]:
my_data_y = my_data_y.dropna()
index = my_data_y[(my_data_y['Dia']=='Acc') | (my_data_y['Dia']=='DIF.(%)') | (my_data_y['Dia']=='Acum') | (my_data_y['Dia']==1) | (my_data_y['PRINCIPAL (kW)']==0)].index.values
my_data_y.drop(index, inplace = True)

index = my_data_x[(my_data_x['Dia']=='Acc') | (my_data_x['Dia']=='DIF.(%)') | (my_data_x['Dia']=='Acum') | (my_data_x['Dia']==1) | (my_data_x['Dia']==0)].index.values
my_data_x.drop(index, inplace = True)

In [ ]:
#my_data = my_data[1:30]
my_data_y['Dia'] = pd.to_datetime(my_data_y['Dia'])
sorted_df_y = my_data_y.sort_values(by='Dia')

my_data_x['Dia'] = pd.to_datetime(my_data_x['Dia'])
sorted_df_x = my_data_x.sort_values(by='Dia')

In [ ]:
my_data = my_data_y.merge(my_data_x, on='Dia', how='outer')
sorted_df = my_data.sort_values(by='Dia')
sorted_df = sorted_df.dropna()
sorted_df = sorted_df.rename(columns = {"Consumo de Água (m³).1":"Consumo de Água"})

Código para gravação de arquivo excel

In [ ]:
sorted_df.to_excel(Files_Folder + 'DataBase.xlsx', index = False)